In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [34]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

In [78]:
def get_chunk(train_data, chunkSize=100):
    
    dummies = pd.get_dummies(train_data['label'], prefix='label', drop_first=False)
    samples = train_data.drop('label', axis=1)
    labels = dummies
    
    datas = pd.concat([labels, samples], axis=1)
    random_batch = datas.loc[np.random.randint(42000, size=chunkSize)]
    images = random_batch.iloc[:, 10:].values
    images = images.astype(np.float)
    images = np.multiply(images, 1.0 / 255.0)    
    labels = random_batch.iloc[:, 0:10].values
    
    return images, labels

# 定义全连接神经网络
class Fcn():
    def __init__(self, in_units=784, h1_units=300):
        self.in_units = in_units
        self.h1_units = h1_units
        
        self.graph = tf.Graph()

        self.train_images = None
        self.train_labels = None
        self.test_images = None
        self.test_labels = None
    
    def define_graph(self):
        with self.graph.as_default():
            # 这里定义图谱中的各种变量
            self.train_images = tf.placeholder(
                tf.float32, shape=(None, self.in_units)
            )
            self.train_labels  = tf.placeholder(
                tf.float32, shape=(None, 10)
            )
            self.test_images  = tf.placeholder(
                tf.float32, shape=(None, self.in_units)
            )
            
            self.w1 = tf.Variable(tf.truncated_normal([self.in_units, self.h1_units], stddev = .1))
            self.b1 = tf.Variable(tf.zeros([self.h1_units]))
            self.w2 =  tf.Variable(tf.truncated_normal([self.h1_units, 10], stddev = .1))
            self.b2 = tf.Variable(tf.zeros([10]))
            
            self.x = tf.placeholder(tf.float32, [None, self.in_units])
            self.keep_prob = tf.placeholder(tf.float32)
        
            def model(x):
                hidden1= tf.nn.relu(tf.matmul(x, self.w1) + self.b1)
                hidden1_drop = tf.nn.dropout(hidden1, self.keep_prob)
                y = tf.nn.softmax(tf.matmul(hidden1_drop, self.w2) + self.b2)
            
                return y
            
            # Training computation. (训练)
            logits = model(self.train_images)
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.train_labels)
            )

            # Optimizer. (优化器)
            self.optimizer = tf.train.AdagradOptimizer(0.01).minimize(self.loss)

            # Predictions for the training, validation, and test data. (预测数据)
            self.train_prediction = tf.nn.softmax(logits)
            self.test_prediction = tf.nn.softmax(model(self.test_images))
    
    def run(self):
        self.session = tf.Session(graph=self.graph)
        with self.session as sess:
            tf.global_variables_initializer().run()
            print("训练开始")
            for i in range(1000):
                samples, labels = get_chunk(train_data)
                sess.run([], feed_dict={
                    self.train_images: samples,
                    self.train_labels: labels,
                    self.keep_prob: 0.75
                })
                
                if(i % 50 == 0):
                    print('{} step'.fromat(i))
            
            print("训练结束")

In [ ]:
f = Fcn()
f.define_graph()
f.run()